<a href="https://colab.research.google.com/github/cda79/IAT360-LLM/blob/main/IAT360_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Required libraries

In [ ]:
!pip install transformers torch datasets

In [ ]:
!pip install transformers torch accelerate

##Import pre-made dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("lanretto/shakespeare-vs-modern-dialogue")
dataset

In [ ]:
## For now, unused
# from datasets import load_dataset
# dataset = load_dataset("madha98/Shakespeare")
# dataset

##Data pre-processing - for training/test split

In [ ]:
#example print
dataset['train'][0]

In [ ]:
#shuffle and split the dataset into a smaller amount
#select only the first 100 and shuffle them up
dataset = dataset['train'].shuffle(seed=30).select(range(100))
dataset

In [ ]:
# create test and train dataset from this shuffled amount
# 80-20 split
dataset = dataset.train_test_split(train_size=0.8, seed=40)
dataset


## Load custom dataset files
For when we make our own, here is some temp code (not to be run)

In [ ]:
#Load files as dataset
data_files={"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Save it to our huggingface hub (not to be run)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

##Tokenization

So this is when we'd split our custom dataset into a third column that combines the features together into one line for training i think